In [3]:
# clean_data.py
import pandas as pd
import numpy as np

STEP   = 0.25
OFFSET = 0.125

def load_data():
    resort_data = pd.read_csv("/Users/haleybrewster/DSP/resorts.csv",encoding="cp1252")
    snow_data = pd.read_csv("/Users/haleybrewster/DSP/snow.csv",encoding="cp1252")
    
    # having coordinates match between data sets pt 1
    europe = (
        resort_data.loc[resort_data["Continent"] == "Europe"]
        .assign(
            latitude  = lambda df: (
                np.round((df["Latitude"] - OFFSET) / STEP) * STEP + OFFSET
            ).round(3),
            longitude = lambda df: (
                np.round((df["Longitude"] - OFFSET) / STEP) * STEP + OFFSET
            ).round(3),
        )
        .drop(columns=[
            "Latitude", "Longitude", "Continent",
            "Chair lifts", "Gondola lifts", "Surface lifts",
            "Child friendly", "Longest run", "ID", "Summer skiing"
        ])
    )
# having coordinates match between data sets pt 2
    snow_data["latitude"]  = snow_data["Latitude"].round(3)
    snow_data["longitude"] = snow_data["Longitude"].round(3)

    snow_mean = (
        snow_data
        .groupby(["latitude", "longitude"], as_index=False)["Snow"]
        .mean()
    )

    # Combine resorts with snow data
    #inner keeps only resorts with matching snow cells
    europe_with_snow = europe.merge(
        snow_mean,
        on=["latitude", "longitude"],
        how="inner"
    )

    return europe_with_snow

if __name__ == "__main__":
    df = load_data()
    df.to_csv("/Users/haleybrewster/DSP/europe_with_snow.csv", index=False)
    print("Saved cleaned data to europe_with_snow.csv")


Saved cleaned data to europe_with_snow.csv
